In [19]:
import openmeteo_requests

import requests_cache
from retry_requests import retry
import polars as pl
from datetime import datetime, timedelta

In [20]:
start_date = '2021-01-01'
end_date = '2025-03-29'

In [21]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables matters
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    # Lat and lon for O'Hare
	"latitude": 41.978611,
	"longitude": -87.904724,
	"start_date": start_date,
	"end_date": end_date,
	"daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "weather_code"],
	"timezone": "America/Chicago",
    'temperature_unit': 'fahrenheit',
    'wind_speed_unit': 'mph',
    'precipitation_unit': 'inch'
}

responses = openmeteo.weather_api(url, params=params)

In [22]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process daily data
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()
daily_weather_code = daily.Variables(5).ValuesAsNumpy()

daily_data = {
	"date": pl.date_range(
        	start=datetime.strptime(start_date, '%Y-%m-%d').date(),
			end=datetime.strptime(end_date, '%Y-%m-%d').date(),
			interval=timedelta(days=1),
			closed='both',
			eager=True
	)
}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["weather_code"] = daily_weather_code

chi_weather_daily_df = pl.DataFrame(daily_data)